# TP Reco
## Import

In [9]:
import numpy as np 
import glob
import csv
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


## Recuperation Données

In [2]:
def read_csv_file(_file_name):
    df=pd.read_csv(_file_name, sep=',', header=None)
    return df.values

## Trouve les fichiers et leurs noms

In [3]:
def init_data_file(hand_='LH', sequence_=None, frame_=None, label_=None):
    data = {hand_ : [{},{},{},{},{},{},{},{},{},{},{}] }
    for name in glob.glob('PositionHandjoints/*'):
        name_file = name.split('\\')[1]
        elems = name_file.split('_')

        d_type = elems[0]
        if hand_ == d_type:
            sequence =  elems[1].split('S')[1]
            if sequence_ == None or sequence_ == sequence:
                frame = elems[2].split('f')[1]
                if frame_ == None or frame_ == frame:
                    length = elems[3].split('l')[1]
                    label = elems[4].split('.')[0]
                    if label_ == None or label_ == label:
                        data[str(d_type)][int(sequence)-1].update({int(frame) : [int(length),label]})

    return data
    
init = init_data_file()

## Lecture de tous les csv et stockage dans un tableau par main

In [98]:
def read_all_files(hand_='LH', sequence_=None, frame_=None, label_=None):
    alldata_file = init_data_file(hand_, sequence_, frame_,label_)
    data = []
    cible = []
    nb_classe_cible = 0
    
    for key,values in alldata_file.items():
        sequence_max = len(values)-1
        for seq in range(sequence_max):
            for key2 in values[seq].keys():
                filename = 'PositionHandJoints/'+ key + '_S' + "%s" % (seq+1) + '_f' + "%s" % key2 + '_l' + "%s" % values[seq][key2][0] + '_' + values[seq][key2][1] + '.csv'
                table = pd.read_csv(filename, sep=' ',header=None)

                val = table.values
                m,d = val.shape
                cible_tmp = [int(nb_classe_cible) for i in range(m)]
                for i in range(m):
                    if i > 0 :
                        cible.append(cible_tmp[i])
                        data_tmp = []
                        for j in range(d-1):
                            data_tmp.append(float(val[i][j])) 
                        data.append(data_tmp)
                nb_classe_cible += 1
    cible = cible[:][:]
    return np.asarray(data),np.asarray(cible), nb_classe_cible

In [105]:
data_left, target_left, nb_left = read_all_files(hand_='LH')
print('Nombre de classes de la main gauche :', nb_left)

data_right, target_right, nb_right = read_all_files(hand_='RH')
print('Nombre de classes de la main droite :', nb_right)

Nombre de classes de la main gauche 223
Nombre de classes de la main droite 377


## Separation data
### Gauche

In [100]:
X_train_left, X_test_left, y_train_left, y_test_left = train_test_split(data_left, target_left, test_size=0.2)
X_train_left = np.asarray(X_train_left)
X_test_left = np.asarray(X_test_left)

### Droite

In [ ]:
X_train_right, X_test_right, y_train_right, y_test_right = train_test_split(data_right, target_right, test_size=0.2)
X_train_right = np.asarray(X_train_right)
X_test_right = np.asarray(X_test_right)

## SVC
### Gauche

In [101]:
svc = SVC(gamma='auto')
svc.fit(X_train_left, y_train_left)
svc.score(X_test_left, y_test_left)

0.9676437000380662

### Droite

In [ ]:
svc = SVC(gamma='auto')
svc.fit(X_train_right, y_train_right)
svc.score(X_test_right, y_test_right)

## Gaussian
### Gauche

In [102]:
gnb = GaussianNB()
gnb.fit(X_train_left, y_train_left)
gnb.score(X_test_left, y_test_left)

0.7921583555386372

### Droite

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train_right, y_train_right)
gnb.score(X_test_right, y_test_right)

## KNN
### Gauche

In [103]:
neigh = KNeighborsClassifier(n_neighbors=nb)
neigh.fit(X_train_left, y_train_left)
neigh.score(X_test_left, y_test_left)

0.7913970308336505

### Droite

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=nb)
neigh.fit(X_train_right, y_train_right)
neigh.score(X_test_right, y_test_right)

## Logistic Regression

In [104]:
lr = LogisticRegression(max_iter=1000).fit(X_train, y_train)
lr.score(X_test, y_test)

0.9965740388275599